# DEBUG: Generate UNet masks from annotations

**22.02.2022**: In pratica la stessa cosa che faccio nel file "generate_unet_annotations.py", per fare un debug

**04.07.2022**: Aggiorno lo script per essere in linea con lo stato attuale del codice. Lo uso per testare il preprocessing delle annotations con la nuova versione del dataset (cioè il dataset che ho corretto io manualmente nelle ultime settimane in modo interattivo usando Napari)

In [1]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

In [2]:
import os

import numpy as np
import napari
import math

from config import config

from data.data_processing_tools import moving_average, keep_percentile
from utils.in_out_tools import load_movies_ids, load_annotations_ids

#### Set working directories

In [3]:
# set working directory
data_dir = os.path.join("..", "data", "raw_data_and_processing")

# set input masks directory and names
original_masks_dir = os.path.join(
    data_dir, "manual_corr_separated_event_masks"
)  # annotation masks folder
class_masks_name = "corrected_label_mask_V3"  # annotation masks to process
event_masks_name = "corrected_rgb_mask_V4"  # event masks to use

# set videos directory
movies_dir = os.path.join(
    data_dir, "smoothed_movies"
)  # videos used to process the annotations

# set output directory
out_dir = os.path.join(
    data_dir, "unet_masks_TEST"
)  # save here masks ready for training
os.makedirs(out_dir, exist_ok=True)

In [4]:
# list of all movie IDs
movie_ids = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
    "13",
    "14",
    "15",
    "16",
    "17",
    "18",
    "19",
    "20",
    "21",
    "22",
    "23",
    "24",
    "25",
    "27",
    "28",
    "29",
    "30",
    "32",
    "33",
    "34",
    "35",
    "36",
    "38",
    "39",
    "40",
    "41",
    "42",
    "43",
    "44",
    "45",
    "46",
]

#### Load input annotation and class masks and videos

In [5]:
# load movies
movies = load_movies_ids(
    data_folder=movies_dir,
    ids=movie_ids,
    names_available=True,
    movie_names="smoothed_video",
)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offs

In [6]:
# load annotation masks
class_masks = load_annotations_ids(
    data_folder=original_masks_dir, ids=movie_ids, mask_names=class_masks_name
)

In [7]:
# load event masks
event_masks = load_annotations_ids(
    data_folder=original_masks_dir, ids=movie_ids, mask_names=event_masks_name
)

## Process movies using k-th percentile method

For each spark annotation mask, compute the k-th percentile over the values of the movie in each event ROI. Then keep only the values in the ROI that are above the percentile.

Ideally: 1 connected component = 1 spark

In [8]:
# compute shape for maximum filter -> min distance between peaks
radius = math.ceil(config.min_dist_xy / 2)
y, x = np.ogrid[-radius : radius + 1, -radius : radius + 1]
disk = x**2 + y**2 <= radius**2
min_dist_filter = np.stack([disk] * (config.min_dist_t + 1), axis=0)
# not the same as config.conn_mask!

### Process sparks for first time, using original smoothed movies

Ideally: 1 connected component = 1 spark

In [9]:
k = 75  # choose value k of percentile
n_iter = 2  # number of dilation/erosion iterations

#### Compute "percentile masks" of a list of movies

In [20]:
# compute new percentile mask for all movies and store them in a dict
percentile_masks = {}

In [26]:
# remaining_movie_ids = ["12","19","20","21"]
remaining_movie_ids = ["07"]

In [27]:
for sample_id in remaining_movie_ids:
    movie = movies[sample_id]
    class_mask = class_masks[sample_id]
    event_mask = event_masks[sample_id]

    # normalise input movie between 0 and 1
    movie = (movie - movie.min()) / (movie.max() - movie.min())

    # get event masks of sparks only
    spark_events_mask = np.where(class_mask == 1, event_mask, 0)

    # get spark events IDs
    spark_events_ids = list(np.unique(spark_events_mask))
    spark_events_ids.remove(0)

    print(f"List of sparks IDs in movie {sample_id}:", *spark_events_ids)

    # new events mask using percentile method
    percentile_events_mask = np.zeros_like(spark_events_mask)

    for spark_id in spark_events_ids:
        event_mask = spark_events_mask == spark_id

        # reduce sparks size dimension wrt to percentile
        new_event_mask = keep_percentile(
            movie=movie,
            roi_mask=event_mask,
            percentile=k,
        )
        percentile_events_mask[new_event_mask] = spark_id

    percentile_masks[sample_id] = percentile_events_mask

List of sparks IDs in movie 07: 3573375 4342810 4385274 7251112 9400685 14726463 16370811


#### Run this to analyse a single mask

In [28]:
sample_id = "07"
movie = movies[sample_id]
class_mask = class_masks[sample_id]
event_mask = event_masks[sample_id]

# normalise input movie between 0 and 1
movie = (movie - movie.min()) / (movie.max() - movie.min())

# get original sparks mask and list of spark IDs
spark_events_mask = np.where(class_mask == 1, event_mask, 0)
spark_events_ids = list(np.unique(spark_events_mask))
spark_events_ids.remove(0)

print("List of sparks IDs:", *spark_events_ids)

# get percentile mask
percentile_events_mask = percentile_masks[sample_id]

List of sparks IDs: 3573375 4342810 4385274 7251112 9400685 14726463 16370811


In [29]:
np.unique(np.where(percentile_events_mask)[0])

array([ 64,  65,  66,  67,  68,  69,  72, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 407, 408, 409, 410, 411, 427, 428,
       429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 479, 480],
      dtype=int64)

In [30]:
viewer = napari.Viewer()
viewer.add_image(movie, name="movie")
viewer.add_labels(spark_events_mask, name="spark events", opacity=0.2)
viewer.add_labels(
    percentile_events_mask, name="percentile-method spark events", opacity=0.9
)
# viewer.add_labels(percentile_events_mask_0, name='percentile-method spark events no erosion/dilation')

<Labels layer 'percentile-method spark events' at 0x20ca6b6bc40>

## 2nd round: compute moving average on smoothed movies where sparks couldn't be correctly separated

Movies that still require processing:

01, 05, 08, 09, 11, 12, 19, 20, 21, 23, 24, 27, 28, 29, 30, 33, 34, 35, 36, 39, 46

In [89]:
movie_ids_2nd_round = [
    "01",
    "05",
    "08",
    "09",
    "11",
    "12",
    "19",
    "20",
    "21",
    "23",
    "24",
    "27",
    "28",
    "29",
    "30",
    "33",
    "34",
    "35",
    "36",
    "39",
    "46",
]

#### Process smoothed movies and compute moving average

In [111]:
# create dict that will contain the averaged movies
averaged_movies = {}

In [155]:
for sample_id in movie_ids_2nd_round:
    movie = movies[sample_id]
    averaged_movies[sample_id] = moving_average(movie, k=3)

#### Compute percentile masks based on averaged movie

In [156]:
# compute new percentile mask for all movies and store them in a dict
avg_percentile_masks = {}

In [157]:
for sample_id in movie_ids_2nd_round:
    movie = averaged_movies[sample_id]
    class_mask = class_masks[sample_id]
    event_mask = event_masks[sample_id]

    # normalise input movie between 0 and 1
    movie = (movie - movie.min()) / (movie.max() - movie.min())

    # get event masks of sparks only
    spark_events_mask = np.where(class_mask == 1, event_mask, 0)

    # get spark events IDs
    spark_events_ids = list(np.unique(spark_events_mask))
    spark_events_ids.remove(0)

    print(f"List of sparks IDs in movie {sample_id}:", *spark_events_ids)

    # new events mask using percentile method
    percentile_events_mask = np.zeros_like(spark_events_mask)

    for spark_id in spark_events_ids:
        event_mask = spark_events_mask == spark_id

        # reduce sparks size dimension wrt to percentile
        new_event_mask = keep_percentile(
            movie=movie,
            roi_mask=event_mask,
            percentile=k,
        )
        percentile_events_mask[new_event_mask] = spark_id

    avg_percentile_masks[sample_id] = percentile_events_mask

List of sparks IDs in movie 01: 5 6 7 1213085 1460377 1763421 3865116 5047384 6923320 7217616 7235208 7607517 9047504 9985923 10065583 10095167 10331602 11014150 13311101 13600822 13703862 15403210 15706005
List of sparks IDs in movie 05: 5 9 9360 114610 135712 422301 463234 633178 744573 882547 965265 1106039 1925128 1926514 2339872 2590452 2601022 2724838 3007235 3203756 4224004 4564197 5047972 5247000 5268606 5472814 5507421 5648379 6023853 6132011 6415686 6579549 7368064 7514576 7522274 8183095 8211204 8454086 8919303 9044279 9201768 9754874 9866835 9872999 9957715 10057818 10489847 11453733 11961978 12050082 13171323 13172604 13633841 13654625 13751860 13897800 14208260 14376843 14638229 14660512 15009103 16166492 16548998
List of sparks IDs in movie 08: 6 8 255 8323328 10739877 16581375
List of sparks IDs in movie 09: 196589 2849917 3943349 5387321 6459205 7130058 8352532 9306704 10816540 11474411 12193461 12518581 12828383 12963785 13413706 13589946 15263586 16276204
List of spa

#### Run this to analyse a single mask

In [256]:
sample_id = "46"
movie = averaged_movies[sample_id]
class_mask = class_masks[sample_id]
event_mask = event_masks[sample_id]

# normalise input movie between 0 and 1
movie = (movie - movie.min()) / (movie.max() - movie.min())

# get original sparks mask and list of spark IDs
spark_events_mask = np.where(class_mask == 1, event_mask, 0)
spark_events_ids = list(np.unique(spark_events_mask))
spark_events_ids.remove(0)

print("List of sparks IDs:", *spark_events_ids)

# get percentile mask
percentile_events_mask = percentile_masks[sample_id]
avg_percentile_events_mask = avg_percentile_masks[sample_id]

List of sparks IDs: 6 871378 2952944 5669422 6240425 9615660 10340821 11307398 11577598 12656347 13871294 14430244 14816150 15021344 15468584


In [257]:
np.unique(np.where(avg_percentile_events_mask)[0])

array([ 23,  24,  25,  26,  27,  28,  60,  61,  62,  63,  64,  65,  66,
       112, 113, 114, 126, 127, 128, 129, 130, 133, 291, 292, 293, 297,
       298, 299, 316, 317, 318, 319, 320, 321, 322, 326, 327, 328, 329,
       469, 470, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483,
       484, 485, 486, 487, 566, 567, 568, 569, 570, 571, 581, 582, 583,
       584, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 641,
       642, 643, 644, 645, 646, 647, 653, 676, 677, 678, 679, 680, 685,
       686, 687, 688, 689, 690, 691, 814, 815, 816, 817, 818, 819, 820,
       821, 822, 823], dtype=int64)

In [258]:
viewer = napari.Viewer()
viewer.add_image(movie, name="averaged movie")
viewer.add_labels(spark_events_mask, name="spark events", opacity=0.2)
viewer.add_labels(
    percentile_events_mask,
    name="percentile-method spark events",
    opacity=0.9,
    visible=False,
)
viewer.add_labels(
    avg_percentile_events_mask,
    name="average percentile-method spark events",
    opacity=0.9,
)

<Labels layer 'average percentile-method spark events' at 0x16606a31100>

In [253]:
# get frames of an event
n_event = 14539191
np.unique(np.where(spark_events_mask == n_event)[0])

array([142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152], dtype=int64)

## Process borders

11.07.2022

Applicare dilation e erosion fa sì che nei bordi manchi la ROI. Per questo motivo provo ad usare la ROI pre-dilation+erosion sui bordi per vedere cosa dà come risultato.

La dimensione dei bordi è 